In [1]:
%env CUDA_VISIBLE_DEVICES=6
%env TRANSFORMERS_CACHE=/mnt/LLM/hub
%env OMP_NUM_THREADS=16

import os
import sys
sys.path.insert(0, '..')

import time
import random
from tqdm.auto import trange
import ipynbname  # pip install ipynbname

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

from src.aq import QuantizedLinear


torch.set_num_threads(16)
torch.backends.cudnn.allow_tf32 = False
torch.backends.cuda.matmul.allow_tf32 = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_loading_dir = '/extra_disk_1/vahe1994/BRRR/layer10.self_attn.q_proj.input_activation.pt'
num_codebooks = 2
nbits_per_codebook = 8
out_group_size = 1
in_group_size = 8
batch_size = 16384
beam_size = 1
rrr_rank = 32
beam_search_epochs = 100
sparsity_regularizer = 0
print_frequency = 10
scale_nbits = 0    # 0 means no scales, 16 means no compression;
codebook_values_nbits = 16  # less than 16 means we quantize codebooks as well
init_max_iter = 100

env: CUDA_VISIBLE_DEVICES=6
env: TRANSFORMERS_CACHE=/mnt/LLM/hub
env: OMP_NUM_THREADS=16


/home/jheuristic/anaconda3/lib/python3.9/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(os.getcwd(), ipynbname.name() + ".ipynb")

# start a new wandb run to track this script
run = wandb.init(
    # set the wandb project where this run will be logged
    dir=os.getcwd(),
    project="AddQuantization-debug",
    entity = "rock-and-roll",
    save_code=True,
    name = f"{ipynbname.name()}_AQ_{num_codebooks=}_{out_group_size=}_{in_group_size=}_{nbits_per_codebook=}_{beam_search_epochs=}",
    settings=wandb.Settings(code_dir="."),
    # track hyperparameters and run metadata
    config={
    "num_codebooks" : num_codebooks,
    "out_group_size": out_group_size,
    "in_group_size": in_group_size,
    "group_size" : out_group_size * in_group_size,
    "batch_size" : batch_size,
    "beam_size" : beam_size,
    "nbits_per_codebook" : nbits_per_codebook,
    "codebook_values_nbits": codebook_values_nbits,
    "scale_nbits": scale_nbits,
    "beam_search_epochs": beam_search_epochs,
    "sparsity_regularizer": sparsity_regularizer,
    "rrr_rank": rrr_rank,
    "init_max_iter": init_max_iter,
    }
)

wandb: Currently logged in as: justheuristic (rock-and-roll). Use `wandb login --relogin` to force relogin


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-70b-hf", torch_dtype='auto', low_cpu_mem_usage=True)

X = torch.load(input_loading_dir, map_location='cpu').float().flatten(0, -2)
reference_weight = model.model.layers[10].self_attn.q_proj.weight.detach().to(device).float()

XTX = torch.zeros(X.shape[-1], X.shape[-1], device=device, dtype=torch.float64)
for i in range(0, len(X), batch_size):
    x_batch = X[i: i + batch_size].cuda().double()
    XTX.addmm_(x_batch.T, x_batch, alpha=1/len(X))
    del x_batch
XTX = XTX.float()
del X

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
quantized_weight = QuantizedLinear(
    XTX=XTX, reference_weight=reference_weight, num_codebooks=num_codebooks,
    nbits_per_codebook=nbits_per_codebook, scale_nbits=scale_nbits, rrr_rank=rrr_rank,
    out_group_size=out_group_size, in_group_size=in_group_size,
    verbose=True, max_iter=init_max_iter,   # faster init, not tested
)
run.log({"Avg_bits": quantized_weight.estimate_nbits_per_parameter()})
print("AVG bits:", quantized_weight.estimate_nbits_per_parameter())
opt = torch.optim.Adam(quantized_weight.parameters(), lr=1e-4, betas=(0.0, 0.95), amsgrad=True)

for epoch in range(1000):
    start = time.perf_counter()
    delta_weight = (quantized_weight() - reference_weight).double()
    loss = (delta_weight @ XTX.double()).flatten() @ delta_weight.flatten() / len(delta_weight)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    run.log({'loss':loss.item()}, step=epoch)
    
    if epoch % print_frequency == 0:
        print(f"loss={loss.item():.10f}\t",
              f"time_on_epoch {epoch} = {time.perf_counter() - start}")
    if (epoch + 1) % beam_search_epochs == 0:
        quantized_weight.beam_search_update_codes_(
            XTX, reference_weight, beam_size=beam_size, sparsity_regularizer=sparsity_regularizer,
            dim_rng=random.Random(), verbose=True)

        if sparsity_regularizer != 0:
            sparsity_rate = ((quantized_weight.codes == 0).sum() / quantized_weight.codes.numel()).item()
            print(f"Sparsity rate {sparsity_rate:.5f}")
            run.log({'sparsity rate': sparsity_rate}, step=epoch)
            mean_code_nbits = sum(get_mean_nbits_by_codebook(quantized_weight.codes)) / num_codebooks
            print(f"mean_code_nbits {mean_code_nbits:.5f}")
            run.log({'Mean codebook length nbits': mean_code_nbits}, step=epoch)
            if in_group_size > 1 and out_group_size > 1:
                curr_avg_bits  = calc_avg_bits(num_codebooks, 1, mean_code_nbits,
                                     nbits_per_codebook, in_features, out_features, scale_nbits)
                run.log({"Avg_bits": curr_avg_bits}, step=epoch)

!!!! RRR RANK = 32


initializing with kmeans:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jheuristic/GPTAQ_rrr/Notebooks/../src/aq.py:570: UserWarning: index_reduce() is in beta and the API may change at any time. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1193.)
  codebook_i, _, _ = fit_kmeans(


AVG bits: 2.1279296875
loss=0.0068659294	 time_on_epoch 0 = 0.47034866688773036
loss=0.0038049441	 time_on_epoch 10 = 0.14076759619638324
loss=0.0032719078	 time_on_epoch 20 = 0.140756756067276


wandb: WARNING Step only supports monotonically increasing values, use define_metric to set a custom x axis. For details see: https://wandb.me/define-metric
wandb: WARNING (User provided step: 0 is less than current step: 1. Dropping entry: {'loss': 0.006865929370745206, '_timestamp': 1704169255.1907425}).


...

  0%|          | 0/2048 [00:00<?, ?it/s]

loss=0.0018531749	 time_on_epoch 800 = 0.14108340675011277
loss=0.0018511775	 time_on_epoch 810 = 0.14083313709124923
loss=0.0018509621	 time_on_epoch 820 = 0.14077292708680034
loss=0.0018508461	 time_on_epoch 830 = 0.14111936604604125
loss=0.0018507620	 time_on_epoch 840 = 0.1409753169864416
loss=0.0018506944	 time_on_epoch 850 = 0.14106252696365118
loss=0.0018506385	 time_on_epoch 860 = 0.14094429602846503
loss=0.0018505932	 time_on_epoch 870 = 0.14078499702736735
loss=0.0018505592	 time_on_epoch 880 = 0.14128152607008815
loss=0.0018505392	 time_on_epoch 890 = 0.1409889874048531


  0%|          | 0/2048 [00:00<?, ?it/s]

loss=0.0018467491	 time_on_epoch 900 = 0.14098746608942747
loss=0.0018450877	 time_on_epoch 910 = 0.140658896882087
loss=0.0018449137	 time_on_epoch 920 = 0.14111529709771276
loss=0.0018448209	 time_on_epoch 930 = 0.14112113695591688
loss=0.0018447540	 time_on_epoch 940 = 0.14079731702804565
loss=0.0018447008	 time_on_epoch 950 = 0.14076189696788788
loss=0.0018446578	 time_on_epoch 960 = 0.1407570568844676
loss=0.0018446243	 time_on_epoch 970 = 0.14077628683298826
loss=0.0018446019	 time_on_epoch 980 = 0.14097715634852648
loss=0.0018445937	 time_on_epoch 990 = 0.14090442704036832


  0%|          | 0/2048 [00:00<?, ?it/s]